<img src="https://camo.githubusercontent.com/9f7c69771104a2df48a040b897a44ad5387f07f77e1e7e5159e68e874fdf8c7f/68747470733a2f2f7777772e6869742e61632e696c2f2e75706c6f61642f61636164656d69632d656e7472657072656e657572736869702f697269732f706172746e6572732f7368656e6b61724c6f676f2e6a7067">

# Final project 
## Lecturer: Eyal Nussbaum
## Student: Lucas Kujawski

## Intro
In the following notebook, we will analyse a dataset describing League of Legends matches.

The dataset contains around 65K rows, containing on each:

* ##### gameId: number
Unique Riot game ID.

* ##### gameDuraton: number
continuous
Game Duration(seconds)

* ##### blueWardPlaced: number
continuous
blue team ward placed counts(Number of warding totems)

* ##### blueWardkills: number
continuous
blue team ward killed counts(Number of warding killed)

* ##### blueTotalMinionKills: number
continuous
blue team kill minion counts (includign jungle)

* ##### blueJungleMinionKills: number
continuous
blue team kill jungle minion counts

* ##### blueTotalHeal: number
continuous
blue team heal amounts

* ##### redWardPlaced: number
continuous
red team ward placed counts(Number of warding totems)

* ##### redWardkills: number
continuous
red team ward killed counts(Number of warding killed)

* ##### redTotalMinionKills: number
continuous
blue team kill minion counts (includign jungle)

* ##### redJungleMinionKills: number
continuous
blue team kill jungle minion counts

* ##### redTotalHeal: number
continuous
red team heal amounts

* ##### FirstBlood: categorical - Red/Blue	
Which team got the first kill of an enemy champion

* ##### FirstTower: categorical - Red/Blue
Which team first destroyed an enemy turret

* ##### FirstBaron: categorical - Red/Blue
Which team first killed Baron Nashor

* ##### FirstDragon: categorical - Red/Blue
Which team forst killed a Dragon

* ##### win: Target Class - Red/Blue
Who won the game


## Initial data analysis

In [1]:
!pip install numpy
# !pip install matplotlib
# !pip install seaborn
!pip install pandas
# !pip install sklearn
# !pip install scipy
# !pip install pydotplus

import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
import pandas as pd
# from sklearn.naive_bayes import GaussianNB
# import datetime
# from pandas.api.types import is_numeric_dtype
# import sklearn as skl
# from scipy.stats import skewnorm
# import scipy.stats as stats
# from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
# from sklearn.model_selection import train_test_split # Import train_test_split function
# from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
# import pydotplus

You should consider upgrading via the '/Users/lucaskujawski/.virtualenvs/data2/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/lucaskujawski/.virtualenvs/data2/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/lucaskujawski/.virtualenvs/data2/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/lucaskujawski/.virtualenvs/data2/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/lucaskujawski/.virtualenvs/data2/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/lucaskujawski/.virtualenvs/data2/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/lucaskujawski/.virtualenvs/data2/bin/python -m pip install --upgrade pip' command.


In [2]:
#definitions
int_columns = ['blueWardPlaced', 'blueWardkills', 'blueTotalMinionKills', 'blueJungleMinionKills',
               'blueTotalHeal', 'redWardPlaced', 'redWardkills', 'redTotalMinionKills',
               'redJungleMinionKills', 'redTotalHeal']
categorical = ['win', 'FirstBlood', 'FirstTower', 'FirstBaron', 'FirstDragon']

In [3]:
df = pd.read_csv('lol3.csv')
df.describe()

,gameId,gameDuraton,blueWardPlaced,blueWardkills,blueTotalMinionKills,blueJungleMinionKills,blueTotalHeal,redWardPlaced,redWardkills,redTotalMinionKills,redJungleMinionKills,redTotalHeal
count,6.589600e+04,65896.000000,65896.000000,59289.000000,65896.000000,65896.000000,65896.000000,65896.000000,50180.000000,59340.000000,54098.000000,63355.000000
mean,4.172854e+09,1427.961591,53.617564,19.819511,498.855333,122.692303,24462.290974,53.786421,19.566780,503.945383,124.647713,24838.933454
std,7.889291e+07,429.784566,31.231714,15.018380,179.961166,67.143071,15150.270579,31.459582,14.823881,183.780133,69.093028,15314.030768
min,3.191955e+09,132.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.144505e+09,1121.000000,33.000000,8.000000,379.000000,81.000000,13446.750000,33.000000,8.000000,381.000000,82.000000,13686.000000
50%,4.194611e+09,1414.000000,54.000000,18.000000,515.000000,126.000000,21448.000000,54.000000,18.000000,521.000000,128.000000,21873.000000
75%,4.224057e+09,1724.000000,74.000000,29.000000,623.000000,169.000000,32149.000000,74.000000,29.000000,632.000000,171.000000,32680.000000
max,4.257022e+09,3301.000000,230.000000,118.000000,1315.000000,400.000000,261707.000000,226.000000,117.000000,1287.000000,1003.000000,206758.000000


* We remove duplicated rows based on gameId. 

In [4]:
df.drop_duplicates(subset=['gameId'], keep='first', inplace=True)

* We replace numeric integer NA values for 0, as we assume that this could be a reasonable value.

In [ ]:
df[int_columns] = df[int_columns].fillna(value=0, axis=1, inplace=False)

* NA Values on win or FirstX invalidates the row as we can't analyse what happen on that game. We remove those lines

In [ ]:
df.dropna(inplace=True, subset=categorical)

* Numeric columns are integer.

In [ ]:
df[int_columns] = df[int_columns].astype('int32')

In [ ]:
df['minutesDuration'] = (df['gameDuraton'] / 60).astype('int32')
df.drop(['gameDuraton'], axis=1, inplace=True)

In [ ]:
df.head()
df.describe()

In [5]:
df.head()
df.describe()

* NA Values on win or FirstX invalidates the row as we can't analyse what happen on that game. We remove those lines

In [6]:
df.dropna(inplace=True, subset=categorical)

* Numeric columns are integer.

In [7]:
df[int_columns] = df[int_columns].astype('int32')

In [13]:
df['minutesDuration'] = (df['gameDuraton'] / 60).astype('int32')
df.drop(['gameDuraton'], axis=1, inplace=True)

In [15]:
df.head()
df.describe()

,gameId,blueWardPlaced,blueWardkills,blueTotalMinionKills,blueJungleMinionKills,blueTotalHeal,redWardPlaced,redWardkills,redTotalMinionKills,redJungleMinionKills,redTotalHeal,minutesDuration
count,4.315000e+04,43150.000000,43150.000000,43150.000000,43150.000000,43150.000000,43150.000000,43150.000000,43150.00000,43150.000000,43150.000000,43150.000000
mean,4.172820e+09,53.620927,17.793326,498.498285,122.700579,24439.601622,53.735944,14.854809,453.14934,102.468158,23857.031518,23.308088
std,7.820261e+07,31.232217,15.420896,180.196161,67.237153,15135.077156,31.421700,15.364916,231.06641,78.839544,15822.796342,7.173454
min,3.191955e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,2.000000
25%,4.144228e+09,33.000000,4.000000,379.000000,81.000000,13446.250000,33.000000,0.000000,316.00000,0.000000,12680.000000,18.000000
50%,4.194388e+09,54.000000,16.000000,514.000000,125.000000,21380.000000,54.000000,12.000000,496.00000,108.000000,21044.500000,23.000000
75%,4.224105e+09,74.000000,28.000000,623.000000,169.000000,32117.000000,74.000000,25.000000,618.00000,161.000000,32114.000000,28.000000
max,4.257021e+09,230.000000,118.000000,1315.000000,400.000000,261707.000000,226.000000,117.000000,1287.00000,1003.000000,143347.000000,55.000000
